## Example with CNN-LSTM on live BTC close price using the full data

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/sats4u/')

Mounted at /content/drive


In [2]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.9 MB/s 
     |████████████████████████████████| 70 kB 9.0 MB/s 
     |████████████████████████████████| 288 kB 60.4 MB/s 
     |████████████████████████████████| 112 kB 81.5 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 78.7 MB/s 
     |████████████████████████████████| 271 kB 81.4 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
     |████████████████████████████████| 144 kB 75.5 MB/s 
     |████████████████████████████████| 749 kB 66.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [3]:
import json
import sys
import pandas as pd
import numpy as np
import math
from datetime import datetime
import time
import matplotlib.pyplot as plt
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow as tf
import src.loadcrypto as lc
import src.featbuild as fb
import src.candles2timeseries as c2t
import src.sats2model as s2m

np.random.seed(420)
os.environ['PYTHONHASHSEED'] = '0'
tf.random.set_seed(420)

## Load Asset Details, Binance client data and Download 15m candles from Binance

In [4]:
data_folder = "data/"
asset_details = pd.read_csv(data_folder + 'asset_details.csv')
secret_data_folder = '/content/drive/MyDrive/Colab Notebooks/crypto_project/crypto_data/'
secrets_filename = secret_data_folder+'data.json'
crypto = lc.CryptoData(asset_details,data_folder)
crypto.load_binance_client(secrets_filename,data1_str = 'DATA1',data2_str = 'DATA2i')
crypto.trade_time_units(dt=60,kline_size="15m",starting_date = '1 Mar 2017')
tickers=crypto.asset_details["Ticker"]
tickers = list(tickers[tickers=='BTCUSDT'].values)
ldata_df = crypto.load_cryptos(tickers,save = False)                             

All caught up..!
size of dataset: (168767, 8)


### Generate Candles and build features from technical analysis

In [5]:
can_btc = fb.Candles(ldata_df,"Bitcoin")
can_btc.buildfeatures()
ts_btc = c2t.Candle2TimeSeries(can_btc.candles, laststeps = 1000, step_back = 48, candle_step_str = "15m",
                lownorm = 0.2, upnorm= 0.8)

ts_btc.candles2ts()
ts_btc.x_candles = np.asarray(ts_btc.x_candles, dtype=np.float32)
ts_btc.x_time = np.asarray(ts_btc.x_time, dtype=np.float32)

Candles Normalized
Extracted last 1000 steps
Generated time-series
Normalized 'candles_norm' with shape : (1000, 9)
Feature data 'x_candles' with size : 952
Feature data with time intervals 'x_time' with size : 952


In [6]:
USE_PRETRAINED = True
PRESAVED_MODELDATE='220615_18_16'
initial_timestamp = pd.Timestamp(datetime.strptime('220615_18_16', '%y%m%d_%H_%M'))

if USE_PRETRAINED:
  model_name = f"model/LSTM_CNN_pretrained_{PRESAVED_MODELDATE}"
else:
  model_name = f"model/LSTM_CNN_model"

loaded_model =  s2m.ModelTimeSeries(ts_btc.x_candles, ts_btc.x_time, ts_btc.y, split_fraction = 0.0)
loaded_model.sats2model()
loaded_model.load_model(model_name)
loaded_model.sats2pred(predict_on_test = False)
preds = ts_btc.denorm(loaded_model.preds)

In [7]:
delta = can_btc.candles.index[1] - can_btc.candles.index[0]
series_dates = can_btc.candles.index.shift(1,freq = delta)
can_btc.candles

,Low,High,Open,vol_diff,Volume,UpperBB,LowerBB,price2volratio,Close
Date,,,,,,,,,
2017-08-17 06:31:00,4319.83,4345.45,4328.65,-0.006737,3.494176,4354.619977,4264.944023,4.808001,4345.45
2017-08-17 06:46:00,4324.35,4345.45,4345.45,-3.324938,0.169238,4345.916804,4286.227196,-124.676491,4324.35
2017-08-17 07:01:00,4316.62,4316.62,4316.62,-0.167697,0.001541,4346.272361,4287.181639,0.000000,4316.62
2017-08-17 07:16:00,4291.38,4291.38,4291.38,0.037377,0.038918,4348.406850,4281.557150,0.000000,4291.38
2017-08-17 07:31:00,4291.39,4300.00,4291.39,0.679983,0.718901,4348.690957,4280.411043,11.976614,4300.00
...,...,...,...,...,...,...,...,...,...
2022-06-15 21:46:00,22018.21,22445.53,22018.21,818.851140,1781.238370,22205.954304,21428.629696,0.109929,22214.02
2022-06-15 22:01:00,22207.10,22590.52,22214.03,981.273880,2762.512250,22452.959990,21314.750010,0.094642,22475.48
2022-06-15 22:16:00,22318.21,22800.00,22475.48,186.146430,2948.658680,22698.482543,21272.165457,0.058077,22646.73


In [14]:
#true_vals = ts_btc.denorm(ts_btc.candles_norm[:,-1])

# plt.plot(preds[0:len(ts_btc.x_candles)-loaded_model.split_point], color='orange', label='predictions')
# plt.plot(true_vals[loaded_model.split_point:len(ts_btc.x_candles)], color='green', alpha=0.5, label='real data')
# plt.legend()
# plt.show()
#can_btc.candles

In [32]:
shifted_time_indexes = can_btc.shift_time_index(can_btc.candles.index,1,can_btc.dtime_index)
series_predicted = pd.Series(preds,index = shifted_time_indexes[-len(preds):],name="preds")
pd.concat([can_btc.candles["Close"],series_predicted], axis=1)


,Close,preds
Date,,
2017-08-17 06:31:00,4345.45,NaN
2017-08-17 06:46:00,4324.35,NaN
2017-08-17 07:01:00,4316.62,NaN
2017-08-17 07:16:00,4291.38,NaN
2017-08-17 07:31:00,4300.00,NaN
...,...,...
2022-06-15 22:01:00,22475.48,19826.667385
2022-06-15 22:16:00,22646.73,19860.159113
2022-06-15 22:31:00,22516.92,19885.850725


,Close,preds
Date,,
2017-08-17 06:31:00,4345.45,NaN
2017-08-17 06:46:00,4324.35,NaN
2017-08-17 07:01:00,4316.62,NaN
2017-08-17 07:16:00,4291.38,NaN
2017-08-17 07:31:00,4300.00,NaN
...,...,...
2022-06-15 22:01:00,22475.48,19826.667385
2022-06-15 22:16:00,22646.73,19860.159113
2022-06-15 22:31:00,22516.92,19885.850725
